In [2]:
import numpy as np 
from sklearn.model_selection import KFold
import pickle as pk 
import os
from os import listdir
import pandas as pd
from os.path import isfile, join
from keras.utils import to_categorical ,Sequence
from sklearn.utils import shuffle
from keras.models import load_model
import numpy as np
import sys
import os
import pickle as pk
import pandas as pd
from os import listdir
from os.path import isfile, join

from keras.utils import to_categorical ,Sequence
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras import backend as K   
pd.options.mode.chained_assignment = None  # default='warn'

Using TensorFlow backend.
/home/mike/Desktop/venv/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:

'''
pd.options.mode.chained_assignment = None  # default='warn'
map_dict = pk.load(open('data/map.pkl' , 'rb'))
df = pd.read_csv('data/train_label.csv') 
audio_martix = np.load('data/mfcc/X_train.npy') 
df_unmanu = df[df['manually_verified'] == 0]
df_unmanu['trans'] = df_unmanu['label'].map(map_dict)
df_unmanu['onehot'] = df_unmanu['trans'].apply(lambda x: to_categorical(x,num_classes=41))
unmanu_veri_idx = df_unmanu.index.values
un_fnames = df_unmanu['fname'].values

X = audio_martix[unmanu_veri_idx]
Y = df_unmanu['onehot'].tolist()
Y = np.array(Y)
Y = Y.reshape(-1 ,41)

fold_path = 'data/semi_produce'
if not os.path.exists(fold_path):
    os.mkdir(fold_path)

np.save( os.path.join(fold_path, 'X_unverified'), X)
np.save( os.path.join(fold_path, 'Y_unverified'), Y)
np.save( os.path.join(fold_path, 'fname_unverified'), un_fnames)

print(df_unmanu.shape)
print(df_unmanu.head(3))
'''

"\npd.options.mode.chained_assignment = None  # default='warn'\nmap_dict = pk.load(open('data/map.pkl' , 'rb'))\ndf = pd.read_csv('data/train_label.csv') \naudio_martix = np.load('data/mfcc/X_train.npy') \ndf_unmanu = df[df['manually_verified'] == 0]\ndf_unmanu['trans'] = df_unmanu['label'].map(map_dict)\ndf_unmanu['onehot'] = df_unmanu['trans'].apply(lambda x: to_categorical(x,num_classes=41))\nunmanu_veri_idx = df_unmanu.index.values\nun_fnames = df_unmanu['fname'].values\n\nX = audio_martix[unmanu_veri_idx]\nY = df_unmanu['onehot'].tolist()\nY = np.array(Y)\nY = Y.reshape(-1 ,41)\n\nfold_path = 'data/semi_produce'\nif not os.path.exists(fold_path):\n    os.mkdir(fold_path)\n\nnp.save( os.path.join(fold_path, 'X_unverified'), X)\nnp.save( os.path.join(fold_path, 'Y_unverified'), Y)\nnp.save( os.path.join(fold_path, 'fname_unverified'), un_fnames)\n\nprint(df_unmanu.shape)\nprint(df_unmanu.head(3))\n"

In [2]:
fold_path = 'data/semi_produce'
X = np.load(os.path.join(fold_path, 'X_unverified.npy'))
Y = np.load(os.path.join(fold_path, 'Y_unverified.npy'))
fname_unverified = np.load(os.path.join(fold_path, 'fname_unverified.npy'))

# create semi.py
# mypath = 'old_model/resnet_verified_refine_nonNormalize'
mypath = 'resnet_verified_refine_nonNormalize_semi'
models = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
print(models)

score = 0.0
for i, m  in  enumerate( models ):
    print('#'*50)
    print('fold:{}'.format(i))

    model = load_model(m)
    result = model.predict(X,verbose = 1 ,batch_size = 64)
    score += result
    
score /= 10

['resnet_verified_refine_nonNormalize_semi/best_semi2_7.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_2.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_8.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_5.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_4.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_1.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_9.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_10.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_3.h5', 'resnet_verified_refine_nonNormalize_semi/best_semi2_6.h5']
##################################################
fold:0
5763/5763 [==============================] - 6s     
##################################################
fold:1
5760/5763 [============================>.] - ETA: 0s##################################################
fold:2
5760/5763 [============================>.] - ETA: 0s##################################################
fold:3
5760/5763

In [3]:
 semi_X_index = np.argwhere(score > 0.8)

In [4]:
# semi_X_index
# score.shape
semi_X_index.shape


(3362, 2)

In [5]:
semi_X = []
semi_Y = []

for l in semi_X_index:
    idx = l[0]
    cate = l[1]
    cate = to_categorical(cate,num_classes=41)
    
    semi_X.append(X[idx])
    semi_Y.append(cate)

semi_X = np.array(semi_X)
semi_Y = np.array(semi_Y)

In [6]:
fold_path = 'data/semi_produce'
np.save(os.path.join(fold_path, 'semi_X_1.npy') , semi_X)
np.save(os.path.join(fold_path, 'semi_Y_1.npy'), semi_Y)

# refine model with semi_data

In [3]:
fold_path = 'data/semi_produce'
semi_X = np.load(os.path.join(fold_path, 'semi_X_1.npy'))
semi_Y = np.load(os.path.join(fold_path, 'semi_Y_1.npy'))

semi_Y = semi_Y.reshape(semi_Y.shape[0],41)

map_dict = pk.load(open('data/map.pkl' , 'rb'))
model_path = 'resnet_verified_refine_nonNormalize_semi'
refine_path = 'resnet_verified_refine_nonNormalize_semi_2'
models = [join(model_path, f) for f in listdir(model_path) if isfile(join(model_path, f))]

if not os.path.exists(refine_path):
    os.mkdir(refine_path)
    
for i in range(2,4):
    
    X_train = np.load('data/ten_fold_data/X_train_{}.npy'.format(i)) 
    Y_train = np.load('data/ten_fold_data/Y_train_{}.npy'.format(i)) 
    X_test = np.load('data/ten_fold_data/X_valid_{}.npy'.format(i))
    Y_test = np.load('data/ten_fold_data/Y_valid_{}.npy'.format(i))

    print('verified data:')
    print(X_train.shape)
    print(Y_train.shape)
    print('semi data')
    print(semi_X.shape)
    print(semi_Y.shape)
    
    
    #append semi data 
    X_train = np.append(X_train,semi_X , axis=0)
    Y_train = np.append(Y_train,semi_Y , axis=0)
    
    # shuffle new data
    X_train , Y_train = shuffle(X_train, Y_train, random_state=0) 


    print('after append semi data:')
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)

    model = load_model(join(model_path,'best_semi2_{}.h5'.format(i)))
    checkpoint = ModelCheckpoint(join(refine_path , 'best_semi3_%d_{val_acc:.5f}.h5'%i), monitor='val_acc', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor="val_acc", mode="max", patience=50)

    print("#"*50)
    print("Fold: ", i)

    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), callbacks=[checkpoint, early],
                        batch_size=128, epochs=10000)

verified data:
(3339, 40, 345, 1)
(3339, 41)
semi data
(3362, 40, 345, 1)
(3362, 41)
after append semi data:
(6701, 40, 345, 1)
(6701, 41)
(371, 40, 345, 1)
(371, 41)
##################################################
Fold:  2
Train on 6701 samples, validate on 371 samples
Epoch 1/10000
6701/6701 [==============================] - 22s - loss: 0.3357 - acc: 0.9516 - val_loss: 7.4782 - val_acc: 0.3881
Epoch 2/10000
6701/6701 [==============================] - 17s - loss: 0.5742 - acc: 0.9039 - val_loss: 3.1218 - val_acc: 0.5930
Epoch 3/10000
6701/6701 [==============================] - 17s - loss: 0.3821 - acc: 0.9561 - val_loss: 1.7185 - val_acc: 0.7251
Epoch 4/10000
6701/6701 [==============================] - 17s - loss: 0.2867 - acc: 0.9790 - val_loss: 1.1776 - val_acc: 0.7978
Epoch 5/10000
6701/6701 [==============================] - 17s - loss: 0.2386 - acc: 0.9907 - val_loss: 1.0808 - val_acc: 0.8302
Epoch 6/10000
6701/6701 [==============================] - 17s - loss: 0.2240 - a

6701/6701 [==============================] - 17s - loss: 0.1825 - acc: 0.9951 - val_loss: 1.1285 - val_acc: 0.8410
Epoch 31/10000
6701/6701 [==============================] - 17s - loss: 0.1745 - acc: 0.9943 - val_loss: 1.0589 - val_acc: 0.8356
Epoch 32/10000
6701/6701 [==============================] - 17s - loss: 0.1563 - acc: 0.9982 - val_loss: 1.2256 - val_acc: 0.8167
Epoch 33/10000
6701/6701 [==============================] - 17s - loss: 0.1566 - acc: 0.9966 - val_loss: 1.0559 - val_acc: 0.8275
Epoch 34/10000
6701/6701 [==============================] - 17s - loss: 0.1537 - acc: 0.9961 - val_loss: 1.1161 - val_acc: 0.8140
Epoch 35/10000
6701/6701 [==============================] - 17s - loss: 0.1726 - acc: 0.9899 - val_loss: 1.1183 - val_acc: 0.8059
Epoch 36/10000
6701/6701 [==============================] - 17s - loss: 0.2150 - acc: 0.9819 - val_loss: 2.2877 - val_acc: 0.6631
Epoch 37/10000
6701/6701 [==============================] - 17s - loss: 0.2208 - acc: 0.9785 - val_loss: 

6701/6701 [==============================] - 18s - loss: 0.1257 - acc: 1.0000 - val_loss: 0.8701 - val_acc: 0.8518
Epoch 64/10000
6701/6701 [==============================] - 18s - loss: 0.1204 - acc: 1.0000 - val_loss: 0.8615 - val_acc: 0.8571
Epoch 65/10000
6701/6701 [==============================] - 17s - loss: 0.1155 - acc: 1.0000 - val_loss: 0.8569 - val_acc: 0.8544
Epoch 66/10000
6701/6701 [==============================] - 18s - loss: 0.1111 - acc: 1.0000 - val_loss: 0.8614 - val_acc: 0.8571
Epoch 67/10000
6701/6701 [==============================] - 17s - loss: 0.1068 - acc: 1.0000 - val_loss: 0.8590 - val_acc: 0.8544
Epoch 68/10000
6701/6701 [==============================] - 17s - loss: 0.1028 - acc: 1.0000 - val_loss: 0.8596 - val_acc: 0.8518
Epoch 69/10000
6701/6701 [==============================] - 17s - loss: 0.0990 - acc: 1.0000 - val_loss: 0.8532 - val_acc: 0.8491
Epoch 70/10000
6701/6701 [==============================] - 17s - loss: 0.0954 - acc: 1.0000 - val_loss: 

6701/6701 [==============================] - 18s - loss: 0.1424 - acc: 0.9987 - val_loss: 0.9690 - val_acc: 0.8383
Epoch 96/10000
6701/6701 [==============================] - 18s - loss: 0.1428 - acc: 0.9979 - val_loss: 0.9753 - val_acc: 0.8464
Epoch 97/10000
6701/6701 [==============================] - 18s - loss: 0.1354 - acc: 0.9979 - val_loss: 1.0132 - val_acc: 0.8518
Epoch 98/10000
6701/6701 [==============================] - 18s - loss: 0.1279 - acc: 0.9994 - val_loss: 0.9104 - val_acc: 0.8518
Epoch 99/10000
6701/6701 [==============================] - 18s - loss: 0.1226 - acc: 0.9990 - val_loss: 0.9032 - val_acc: 0.8518
Epoch 100/10000
6701/6701 [==============================] - 18s - loss: 0.1157 - acc: 0.9999 - val_loss: 0.8899 - val_acc: 0.8464
Epoch 101/10000
6701/6701 [==============================] - 18s - loss: 0.1113 - acc: 1.0000 - val_loss: 0.8855 - val_acc: 0.8518
Epoch 102/10000
6701/6701 [==============================] - 18s - loss: 0.1075 - acc: 1.0000 - val_los

6701/6701 [==============================] - 18s - loss: 0.1327 - acc: 0.9982 - val_loss: 0.9260 - val_acc: 0.8302
Epoch 128/10000
6701/6701 [==============================] - 18s - loss: 0.1242 - acc: 0.9984 - val_loss: 0.9221 - val_acc: 0.8275
Epoch 129/10000
6701/6701 [==============================] - 18s - loss: 0.1184 - acc: 0.9996 - val_loss: 0.9210 - val_acc: 0.8464
Epoch 130/10000
6701/6701 [==============================] - 16s - loss: 0.1133 - acc: 0.9996 - val_loss: 0.9058 - val_acc: 0.8275
Epoch 131/10000
6701/6701 [==============================] - 16s - loss: 0.1074 - acc: 1.0000 - val_loss: 0.9185 - val_acc: 0.8329
Epoch 132/10000
6701/6701 [==============================] - 16s - loss: 0.1033 - acc: 1.0000 - val_loss: 0.9099 - val_acc: 0.8383
Epoch 133/10000
6701/6701 [==============================] - 16s - loss: 0.0997 - acc: 1.0000 - val_loss: 0.9385 - val_acc: 0.8491
Epoch 134/10000
6701/6701 [==============================] - 16s - loss: 0.0960 - acc: 1.0000 - val

6701/6701 [==============================] - 17s - loss: 0.1231 - acc: 0.9948 - val_loss: 1.1439 - val_acc: 0.8005
Epoch 161/10000
6701/6701 [==============================] - 17s - loss: 0.1184 - acc: 0.9957 - val_loss: 1.1736 - val_acc: 0.8248
Epoch 162/10000
6701/6701 [==============================] - 17s - loss: 0.1152 - acc: 0.9960 - val_loss: 1.2374 - val_acc: 0.7871
verified data:
(3339, 40, 345, 1)
(3339, 41)
semi data
(3362, 40, 345, 1)
(3362, 41)
after append semi data:
(6701, 40, 345, 1)
(6701, 41)
(371, 40, 345, 1)
(371, 41)
##################################################
Fold:  3
Train on 6701 samples, validate on 371 samples
Epoch 1/10000
6701/6701 [==============================] - 23s - loss: 0.3580 - acc: 0.9422 - val_loss: 14.9317 - val_acc: 0.0566
Epoch 2/10000
6701/6701 [==============================] - 18s - loss: 0.5674 - acc: 0.9023 - val_loss: 3.3313 - val_acc: 0.6092
Epoch 3/10000
6701/6701 [==============================] - 18s - loss: 0.3678 - acc: 0.955

6701/6701 [==============================] - 17s - loss: 0.1867 - acc: 0.9916 - val_loss: 1.1261 - val_acc: 0.8329
Epoch 29/10000
6701/6701 [==============================] - 17s - loss: 0.1744 - acc: 0.9940 - val_loss: 0.8808 - val_acc: 0.8410
Epoch 30/10000
6701/6701 [==============================] - 17s - loss: 0.1650 - acc: 0.9960 - val_loss: 1.0367 - val_acc: 0.8167
Epoch 31/10000
6701/6701 [==============================] - 17s - loss: 0.1959 - acc: 0.9893 - val_loss: 1.5369 - val_acc: 0.7682
Epoch 32/10000
6701/6701 [==============================] - 17s - loss: 0.2034 - acc: 0.9851 - val_loss: 1.4315 - val_acc: 0.7736
Epoch 33/10000
6701/6701 [==============================] - 17s - loss: 0.2590 - acc: 0.9712 - val_loss: 1.4456 - val_acc: 0.7736
Epoch 34/10000
6701/6701 [==============================] - 17s - loss: 0.2153 - acc: 0.9855 - val_loss: 0.9829 - val_acc: 0.8383
Epoch 35/10000
6701/6701 [==============================] - 16s - loss: 0.1838 - acc: 0.9927 - val_loss: 

6701/6701 [==============================] - 16s - loss: 0.1852 - acc: 0.9918 - val_loss: 1.0841 - val_acc: 0.8356
Epoch 62/10000
6701/6701 [==============================] - 16s - loss: 0.1686 - acc: 0.9955 - val_loss: 0.9604 - val_acc: 0.8544
Epoch 63/10000
6701/6701 [==============================] - 16s - loss: 0.1604 - acc: 0.9964 - val_loss: 0.9070 - val_acc: 0.8518
Epoch 64/10000
6701/6701 [==============================] - 16s - loss: 0.2010 - acc: 0.9854 - val_loss: 1.1563 - val_acc: 0.8086
Epoch 65/10000
6701/6701 [==============================] - 16s - loss: 0.1953 - acc: 0.9870 - val_loss: 1.0296 - val_acc: 0.8383
Epoch 66/10000
6701/6701 [==============================] - 16s - loss: 0.2508 - acc: 0.9740 - val_loss: 1.5183 - val_acc: 0.7871
Epoch 67/10000
6701/6701 [==============================] - 16s - loss: 0.1726 - acc: 0.9940 - val_loss: 1.0406 - val_acc: 0.8437
Epoch 68/10000
6701/6701 [==============================] - 16s - loss: 0.1776 - acc: 0.9916 - val_loss: 

6701/6701 [==============================] - 16s - loss: 0.1397 - acc: 0.9991 - val_loss: 0.8094 - val_acc: 0.8571
Epoch 93/10000
6701/6701 [==============================] - 16s - loss: 0.1333 - acc: 0.9996 - val_loss: 0.8358 - val_acc: 0.8706
Epoch 94/10000
6701/6701 [==============================] - 16s - loss: 0.1266 - acc: 0.9999 - val_loss: 0.8318 - val_acc: 0.8787
Epoch 95/10000
6701/6701 [==============================] - 16s - loss: 0.1221 - acc: 0.9997 - val_loss: 0.8624 - val_acc: 0.8868
Epoch 96/10000
6701/6701 [==============================] - 16s - loss: 0.1170 - acc: 1.0000 - val_loss: 0.8424 - val_acc: 0.8841
Epoch 97/10000
6701/6701 [==============================] - 16s - loss: 0.1127 - acc: 1.0000 - val_loss: 0.8145 - val_acc: 0.8814
Epoch 98/10000
6701/6701 [==============================] - 16s - loss: 0.1085 - acc: 1.0000 - val_loss: 0.8133 - val_acc: 0.8814
Epoch 99/10000
6701/6701 [==============================] - 16s - loss: 0.1047 - acc: 1.0000 - val_loss: 

6701/6701 [==============================] - 16s - loss: 0.1101 - acc: 0.9967 - val_loss: 1.0537 - val_acc: 0.8275
Epoch 124/10000
6701/6701 [==============================] - 16s - loss: 0.1303 - acc: 0.9903 - val_loss: 1.3844 - val_acc: 0.7574
Epoch 125/10000
6701/6701 [==============================] - 16s - loss: 0.2409 - acc: 0.9597 - val_loss: 2.5834 - val_acc: 0.6631
Epoch 126/10000
6701/6701 [==============================] - 16s - loss: 0.2740 - acc: 0.9527 - val_loss: 1.8258 - val_acc: 0.7197
Epoch 127/10000
6701/6701 [==============================] - 16s - loss: 0.2075 - acc: 0.9767 - val_loss: 1.6290 - val_acc: 0.7790
Epoch 128/10000
6701/6701 [==============================] - 16s - loss: 0.1686 - acc: 0.9867 - val_loss: 0.9501 - val_acc: 0.8598
Epoch 129/10000
6701/6701 [==============================] - 16s - loss: 0.1442 - acc: 0.9939 - val_loss: 0.8615 - val_acc: 0.8491
Epoch 130/10000
6701/6701 [==============================] - 16s - loss: 0.1257 - acc: 0.9984 - val